# Prediction of origin for double electron event
This notebook aims to build a model able to predict where a two electron events originated.

## TODO / Current status
Best metrics so far is an R2-score of ~0.4, with the two first convolutional layers
and activations from the CIFAR-10 example, and only one dense layer (Dense(4)).
Results do not improve by including the entire model, but this might  change
with more date (currently just 5000 double-event for training).
However, the model seems to favour predicting positions at the midpoint between events.
Does the model need to know that (x1,y1) are connected, and (x2,y2) are connected?
Perhaps they should be treated with separate loss? (Isn't the loss already separate for each output feature?)

In [1]:
"""
Here are the data files.  For all data files each image and label is on one row.  
The first 256 values in each row correspond to the 16x16 detector image and 
the last 6 values correspond to Energy1, Xpos1, Ypos1, Energy2, Xpos2, Ypos2.  
If there is no second particle then Energy2 = 0 and Xpos2 and Ypos2 are both -100.  
(When I run my model, I have to reset the -100 to 0).
 
CeBr10kSingle are 10,000 rows of data and labels for single interactions in the detector
CeBr10k_1.txt is 10,000 rows of data and labels with a mix of single interactions and double interactions
CeBr10.txt is a small file I use which contains 10 single interactions.
"""

import numpy as np
import matplotlib.pyplot as plt
from data_functions import separate_simulated_data, label_simulated_data

# File import
PATH = "../data/small_sample/"
filenames = ["CeBr10kSingle_1.txt", "CeBr10kSingle_2.txt", "CeBr10k_1.txt", "CeBr10.txt"]

## single, mix, small define which dataset to load.
file_to_load = "mix"

if file_to_load == "single_1":
    infile = PATH+filenames[0]
if file_to_load == "single_2":
    infile = PATH+filenames[1]
if file_to_load == "mix":
    infile = PATH+filenames[2]
if file_to_load == "small":
    infile = PATH+filenames[3]
if file_to_load == "combined_single":
    infile = PATH+filenames[0]
    infile2 = PATH+filenames[1]

data = np.loadtxt(infile)

if file_to_load == "combined_single":
    data2 = np.loadtxt(infile2)
    data = np.concatenate((data, data2))
    

# Extract the data containing double events. 
# data[:, -3] is Energy2, which is 0 if the data is a single event
double_events = data[np.where(data[:,-3] != 0)]

images, energies, positions = separate_simulated_data(double_events)


print("Image data shape: {}".format(images.shape))
print("Energies shape: {}".format(energies.shape))
print("Positions shape: {}".format(positions.shape))
    

Image data shape: (4998, 16, 16, 1)
Energies shape: (4998, 2)
Positions shape: (4998, 4)



## Set up training and test data

In [2]:
from sklearn.model_selection import train_test_split

# Split the data into training and test sets
x_train, x_test, y_train, y_test = train_test_split(images, positions, test_size = 0.1)
print("Training and test data shapes:")
print("x_train: {}".format(x_train.shape))
print("x_test: {}".format(x_test.shape))
print("y_train: {}".format(y_train.shape))
print("y_test: {}".format(y_test.shape))




Training and test data shapes:
x_train: (4498, 16, 16, 1)
x_test: (500, 16, 16, 1)
y_train: (4498, 4)
y_test: (500, 4)


## Build and compile model
Using Keras as our framework with Tensorflow backend

In [3]:
from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten, Activation
from keras.layers import Conv2D, MaxPooling2D
from keras import backend

# Set up sequetial model based on Keras CIFAR-10 example
model = Sequential()

# Add layers
model.add(Conv2D(32, (3, 3), padding='same',
                 input_shape=x_train.shape[1:]))
model.add(Activation('relu'))
model.add(Conv2D(32, (3, 3)))
model.add(Activation('relu'))
#model.add(MaxPooling2D(pool_size=(2, 2)))
#model.add(Dropout(0.25))

#model.add(Conv2D(64, (3, 3), padding='same'))
#model.add(Activation('relu'))
#model.add(Conv2D(64, (3, 3)))
#model.add(Activation('relu'))
#model.add(MaxPooling2D(pool_size=(2, 2)))
#model.add(Dropout(0.25))

model.add(Flatten())
#model.add(Dense(512))
#model.add(Activation('relu'))
#model.add(Dropout(0.5))
model.add(Dense(4))
model.add(Activation('linear'))



# Custom definition of R2 score for metrics
def r2_keras(y_true, y_pred):
    SS_res =  backend.sum(backend.square(y_true - y_pred)) 
    SS_tot = backend.sum(backend.square(y_true - backend.mean(y_true))) 
    return ( 1 - SS_res/(SS_tot + backend.epsilon()) )

# Compile model
model.compile(loss='mse',
              optimizer='adam',
              metrics=[r2_keras])

# Output summary of model
print(model.summary())


Using TensorFlow backend.
/home/ulvik/.local/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/home/ulvik/.local/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/home/ulvik/.local/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/home/ulvik/.local/lib/python3.7/site-packages/tensorflow/python/fra

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_1 (Conv2D)            (None, 16, 16, 32)        320       
_________________________________________________________________
activation_1 (Activation)    (None, 16, 16, 32)        0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 14, 14, 32)        9248      
_________________________________________________________________
activation_2 (Activation)    (None, 14, 14, 32)        0         
_________________________________________________________________
flatten_1 (Flatten)          (None, 6272)              0         
_________________________________________________________________
dense_1 (Dense)              (None, 4)                 25092     
_________________________________________________________________
activation_3 (Activation)    (None, 4)                 0         
Total para

## Train the model
We also output the mean squared error and R2-score as evaluation metrics

In [5]:
# Parameters for the model
batch_size = 32
epochs = 25

history = model.fit(x_train, y_train,
              batch_size=batch_size,
              epochs=epochs,
              validation_data=(x_test, y_test),
              shuffle=True)

Train on 4498 samples, validate on 500 samples
Epoch 1/25
4498/4498 [==============================] - 2s 490us/step - loss: 9.5113 - r2_keras: 0.4627 - val_loss: 12.7820 - val_r2_keras: 0.2995
Epoch 2/25
 480/4498 [==>...........................] - ETA: 2s - loss: 9.5586 - r2_keras: 0.4706

KeyboardInterrupt: 

## Predict on test-set and plot some examples
To compare predicted positions with actual positions. 

In [ ]:
predicted_pos = model.predict(x_test)

In [ ]:


# Plot some images, with electron origin positions
%matplotlib inline

index = 100
fig, ax = plt.subplots(3, 3, sharex='col', sharey='row', figsize=(12,12))

# Reshape test-data for plotting
x_plot = x_test.reshape(x_test.shape[0], x_test.shape[1], x_test.shape[2])
for i in range(3):
    for j in range(3):
        # plot image
        ax[i, j].imshow(x_plot[index + i*3 + j])
        
        # plot true origin of event
        x1 = y_test[index + i*3 + j][0]
        y1 = y_test[index + i*3 + j][1]
        x2 = y_test[index + i*3 + j][2]
        y2 = y_test[index + i*3 + j][3]
        ax[i, j].plot(x1, y1, 'rx')
        ax[i, j].plot(x2, y2, 'rx')
        
        # plot predicted origin of event
        x_pred1 = predicted_pos[index + i*3 + j][0]
        y_pred1 = predicted_pos[index + i*3 + j][1]
        x_pred2 = predicted_pos[index + i*3 + j][2]
        y_pred2 = predicted_pos[index + i*3 + j][3]
        ax[i, j].plot(x_pred1, y_pred1, 'wx')
        ax[i, j].plot(x_pred2, y_pred2, 'wx')
plt.show()

## Plot metrics history for training and test

In [ ]:
%matplotlib inline

plt.plot(history.history['r2_keras'])
plt.plot(history.history['val_r2_keras'])
plt.title('model R2 score')
plt.ylabel('R2 score')
plt.xlabel('epoch')
plt.axis(ymin=0, ymax=1) # limit to view the interesting part of R2
plt.legend(['train', 'test'], loc='upper left')
plt.show()

# summarize history for loss
plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.title('model loss (mse)')
plt.ylabel('loss')
plt.xlabel('epoch')
plt.legend(['train', 'test'], loc='upper left')
plt.show()